In [ ]:
import pandas as pd
import sklearn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train_data = pd.read_csv('TRAIN.csv')
train_data

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
712041,342,2,2013-10-01,10182,836,1,0,0,0
712042,343,2,2013-10-01,5352,588,1,0,0,0
712043,344,2,2013-10-01,6652,1043,1,0,0,1
712044,345,2,2013-10-01,3842,475,1,0,0,0


In [ ]:
train_data['StateHoliday'].unique()

array(['0', 'a', 'b', 'c', 0], dtype=object)

In [ ]:
train_data['StateHoliday'] = train_data['StateHoliday'].replace([0],'0')

In [ ]:
train_data['StateHoliday'].unique()

array(['0', 'a', 'b', 'c'], dtype=object)

In [ ]:
train_data['SchoolHoliday'].unique()

array([1, 0])

In [ ]:
train_data.isna().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [ ]:
train_data = train_data.dropna()


In [ ]:
train_data.isna().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [ ]:
X = train_data.drop(['Sales','Customers','Date','Store'],axis=1)
Y = train_data['Sales']
Y

0          5263
1          6064
2          8314
3         13995
4          4822
          ...  
712041    10182
712042     5352
712043     6652
712044     3842
712045     6143
Name: Sales, Length: 712046, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_valid,Y_train,Y_valid = train_test_split(X,Y,test_size=0.2,random_state=0)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, ['DayOfWeek','StateHoliday'])
    ])

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# model_1 = RandomForestRegressor(n_estimators=200)

In [ ]:
from xgboost import XGBRegressor

model_1 = XGBRegressor(n_estimators=1000, learning_rate=0.05)


In [ ]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model_1', model_1)
                             ])

my_pipeline.fit(X_train,Y_train)

[17:42:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('cat',
                                                  Pipeline(memory=None,
                                                           steps=[('onehot',
                                                                   OneHotEncoder(categories='auto',
                                                                                 drop=None,
                                                                                 dtype=<class 'numpy.float64'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse=True))],
                                                

In [ ]:
y_train_pred = my_pipeline.predict(X_train)
y_valid_pred = my_pipeline.predict(X_valid)

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(Y_train,y_train_pred))

print("The model performance for training set")
print('RMSE is {}'.format(rmse))
print("\n")

rmse = np.sqrt(mean_squared_error(Y_valid,y_valid_pred))

print("The model performance for training set")
print('RMSE is {}'.format(rmse))
print("\n")

The model performance for training set
RMSE is 2860.601730239405


The model performance for training set
RMSE is 2857.0731316663455




In [ ]:
test_data = pd.read_csv('TEST.csv')
test_data = test_data.drop(['Date','Store'],axis=1)
test_data

,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday
0,1,0,0,b,1
1,6,1,0,0,0
2,4,1,1,0,1
3,4,1,0,0,1
4,2,1,0,0,1
...,...,...,...,...,...
305158,2,1,1,0,1
305159,6,1,0,0,0
305160,1,1,0,0,0
305161,6,1,0,0,0


In [ ]:
y_test_pred = my_pipeline.predict(test_data)

In [ ]:
index = [i for i in range(0,305163)]

In [ ]:
output = pd.DataFrame(data=y_test_pred,columns=['Sales'])
output

,Sales
0,223.464355
1,5997.444336
2,6844.090332
3,6844.090332
4,7202.517578
...,...
305158,7202.517578
305159,5997.444336
305160,8393.502930
305161,5997.444336


In [ ]:
output_csv = output.to_csv('output.csv',index=True)

In [ ]:
df = pd.read_csv('output.csv')
df.head()

,Unnamed: 0,Sales
0,0,223.46436
1,1,5997.44430
2,2,6844.09030
3,3,6844.09030
4,4,7202.51760
